In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [12, 6]

In [2]:
data_path = "../data/origineel"

if not Path(data_path).is_dir():
    raise Warning("Data path does not exist")

In [4]:
data_file_paths = [p for p in Path(data_path).iterdir() if p.suffix==".csv"]

In [7]:
cableconfig = pd.read_csv(data_file_paths[0], sep=";")
partialdischarges = pd.read_csv(data_file_paths[1], sep=";")
warning = pd.read_csv(data_file_paths[2], sep=";")

In [21]:
# Namen van de kolommen
time_col, location_col, charge_col = partialdischarges.columns

# Check that each row either has location and charge undefinied, or both defined:
for i in np.arange(len(partialdischarges))[np.isnan(partialdischarges[location_col]) != np.isnan(partialdischarges[charge_col])]:
    raise Warning(f"Row {i} has missing values")

# Lijst met bools die aangeeft of er op dat tijdstip een PD was
pd_occured = ~np.isnan(partialdischarges[location_col])

# De drie kolommen; alleen op tijdstippen met PD
locations = partialdischarges[location_col][pd_occured]
charges = partialdischarges[charge_col][pd_occured]
times = partialdischarges[time_col][pd_occured].apply(datetime.datetime.fromisoformat)

# Het algoritme

Het algoritme is gebaseerd op DBSCAN (https://en.wikipedia.org/wiki/DBSCAN#Algorithm)

In [76]:
def distance(x,y):
    return abs(x-y)

def neighbors(DB, distfunc, Q, eps):
    neighborhood=[]
    for P in DB: 
        if distance(Q,P)<eps:
            neighborhood.append(P)
    return neighborhood

def DBSCAN(DB, distfunc, eps, minPts):
    C = []
    noise = []
    undefined = []
    for p in DB:
        undefined.append(p)
    for p in DB:
        if p not in undefined:
            continue
        N = neighbors(DB, distfunc, p, eps)
        if len(N) < minPts:
            noise.append(p)
            undefined.remove(p)
            continue
        C.append(p)
        undefined.remove(p)
        S = N
        S.remove(p)
        for q in S:
            if q in noise:
                noise.remove(q)
                C.append(q)
            if q not in undefined:
                continue
            C.append(q)
            undefined.remove(q)
            N = neighbors(DB, distfunc, q, eps)
            if len(N) > minPts:
                S = list(set().union(S,N))
        print(sum(C)/len(C))
        print("This is a cluster")
        C = []
    print(noise) 
    print("This is noise")
    
                
                
        
    

In [79]:
DBSCAN(locations, distance, 10, 10)

1308.787313988871
This is a cluster
818.3295035789974
This is a cluster
796.6306182397396
This is a cluster
829.1336977415897
This is a cluster
862.2187356776685
This is a cluster
846.8789870347656
This is a cluster
581.980141600288
This is a cluster
1321.1509324683188
This is a cluster
884.8580030854876
This is a cluster
561.1722031218559
This is a cluster
639.9041053181296
This is a cluster
783.6966689458653
This is a cluster
722.1207138986377
This is a cluster
853.1721653194501
This is a cluster
752.5093690518152
This is a cluster
348.2259676918575
This is a cluster
368.51673409766374
This is a cluster
484.79216073901046
This is a cluster
386.261517230908
This is a cluster
332.6580371436856
This is a cluster
250.34350458603086
This is a cluster
420.9989490408109
This is a cluster
469.61502422068776
This is a cluster
262.63469328660705
This is a cluster
91.53099074406568
This is a cluster
435.0872281066755
This is a cluster
1.8886869388435927
This is a cluster
1332.8432844920999
This

Dit duurde 8 minuten. De parameters zijn willekeurig gekozen, dus het resultaat zal waarschijnlijk niet kloppen. 